In [ ]:
# Work 08
## Applying written functions to calculate mask properties to all mask output
## Applying trained classification model to predict object group (label group) of each mask based on calculated mask properties
## Calculating entropy variables (image entropy, whole scene entropy and ground scene entropy)
## Summarising all entropy variables for subsequent analysis

In [1]:
# loading libraries
import torch
torch.cuda.empty_cache() 
import google_streetview
import google_streetview.api
import google_streetview.helpers
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2
import urllib.request
from PIL import Image
import pandas as pd
import pyproj
from pathlib import Path
from scipy.stats import entropy
import seaborn as sn

In [7]:
# loading written functions
from mtp_function_yl import *

### Mask properties calculation - accident locations

In [8]:
# loading GSV images
koord  = pd.read_csv('/home/yelhe/script/mt/output/koord.csv')
name1 = koord.Nr.astype(str) + '1.jpg'
name2 = koord.Nr.astype(str) + '2.jpg'
name3 = koord.Nr.astype(str) + '3.jpg'
name4 = koord.Nr.astype(str) + '4.jpg'
gsv_image1 = []
gsv_image2 = []
gsv_image3 = []
gsv_image4 = []
nf1 = []
nf2 = []
nf3 = []
nf4 = []
for i in range(0,349):
    path1 = Path('/home/yelhe/data/gsv/GSV_filtered/' + name1[i])
    c1 = path1.is_file()
    path2 = Path('/home/yelhe/data/gsv/GSV_filtered/' + name2[i])
    c2 = path2.is_file()
    path3 = Path('/home/yelhe/data/gsv/GSV_filtered/' + name3[i])
    c3 = path3.is_file()
    path4 = Path('/home/yelhe/data/gsv/GSV_filtered/' + name4[i])
    c4 = path4.is_file()
    if c1 is True:
        im1 = cv2.imread("/home/yelhe/data/gsv/GSV_filtered/" + name1[i])
        im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2RGB)
        gsv_image1.append(im1)
        nf1.append(name1[i])
    if c2 is True:
        im2 = cv2.imread("/home/yelhe/data/gsv/GSV_filtered/" + name2[i])
        im2 = cv2.cvtColor(im2, cv2.COLOR_BGR2RGB)
        gsv_image2.append(im2)
        nf2.append(name2[i])
    if c3 is True:
        im3 = cv2.imread("/home/yelhe/data/gsv/GSV_filtered/" + name3[i])
        im3 = cv2.cvtColor(im3, cv2.COLOR_BGR2RGB)
        gsv_image3.append(im3)
        nf3.append(name3[i])
    if c4 is True:
        im4 = cv2.imread("/home/yelhe/data/gsv/GSV_filtered/" + name4[i])
        im4 = cv2.cvtColor(im4, cv2.COLOR_BGR2RGB)
        gsv_image4.append(im4)
        nf4.append(name4[i])

# loading filtered segmentation output masks 
n1 = [s.replace('.jpg', '') for s in nf1]
n2 = [s.replace('.jpg', '') for s in nf2]
n3 = [s.replace('.jpg', '') for s in nf3]
n4 = [s.replace('.jpg', '') for s in nf4]
m1f = []
m2f = []
m3f = []
m4f = []
for i in range(0,295):
    m1 = np.load('/home/yelhe/data/gsv_output_filtered_new/' + n1[i] + 'f.npy', allow_pickle=True)
    m1f.append(m1)
    m2 = np.load('/home/yelhe/data/gsv_output_filtered_new/' + n2[i] + 'f.npy', allow_pickle=True)
    m2f.append(m2)
    m3 = np.load('/home/yelhe/data/gsv_output_filtered_new/' + n3[i] + 'f.npy', allow_pickle=True)
    m3f.append(m3)
    m4 = np.load('/home/yelhe/data/gsv_output_filtered_new/' + n4[i] + 'f.npy', allow_pickle=True)
    m4f.append(m4)

In [16]:
# applying feature_summary function to calculate mask properties
df = pd.DataFrame()
df1 = []
df2 = []
df3 = []
df4 = []
for i in range(0,295):
    df1 = feature_summary(gsv_image1[i], m1f[i])
    df1 = df1.assign(Nr = n1[i])
    
    df2 = feature_summary(gsv_image2[i], m2f[i])
    df2 = df2.assign(Nr = n2[i])
    
    df3 = feature_summary(gsv_image3[i], m3f[i])
    df3 = df3.assign(Nr = n3[i])
    
    df4 = feature_summary(gsv_image4[i], m4f[i])
    df4 = df4.assign(Nr = n4[i])
    
    df = pd.concat([df, df1, df2, df3, df4])
    print(i, len(df.index))

# saving mask properties of all accident locations
df.to_csv('/home/yelhe/script/mt/output/df_accident_0_295.csv', index = False)

### Mask properties calculation - random pseudo locations

In [ ]:
# loading GSV images
rp_koord = pd.read_csv('/home/yelhe/script/mt/output/rp_koord_new.csv')
rp_koord['id'] = rp_koord.id.astype(str)
rp_koord['id'] = rp_koord['id'].str.zfill(4)
name1 = rp_koord.id.astype(str) + '1'
name2 = rp_koord.id.astype(str) + '2'
name3 = rp_koord.id.astype(str) + '3'
name4 = rp_koord.id.astype(str) + '4'
gsv_imager1 = []
gsv_imager2 = []
gsv_imager3 = []
gsv_imager4 = []
nrf1 = []
nrf2 = []
nrf3 = []
nrf4 = []
for i in range(0,995):
    path1 = Path('/home/yelhe/data/gsv_rp_f/gsv_rp_nf/' + name1[i] + '.jpg')
    c1 = path1.is_file()
    path2 = Path('/home/yelhe/data/gsv_rp_f/gsv_rp_nf/' + name2[i] + '.jpg')
    c2 = path2.is_file()
    path3 = Path('/home/yelhe/data/gsv_rp_f/gsv_rp_nf/' + name3[i] + '.jpg')
    c3 = path3.is_file()
    path4 = Path('/home/yelhe/data/gsv_rp_f/gsv_rp_nf/' + name4[i] + '.jpg')
    c4 = path4.is_file()
    if c1 is True:
        im1 = cv2.imread("/home/yelhe/data/gsv_rp_f/gsv_rp_nf/" + name1[i] + ".jpg")
        gsv_imager1.append(im1)
        nrf1.append(name1[i])
    if c2 is True:
        im2 = cv2.imread("/home/yelhe/data/gsv_rp_f/gsv_rp_nf/" + name2[i] + ".jpg")
        gsv_imager2.append(im2)
        nrf2.append(name2[i])
    if c3 is True:
        im3 = cv2.imread("/home/yelhe/data/gsv_rp_f/gsv_rp_nf/" + name3[i] + ".jpg")
        gsv_imager3.append(im3)
        nrf3.append(name3[i])
    if c4 is True:
        im4 = cv2.imread("/home/yelhe/data/gsv_rp_f/gsv_rp_nf/" + name4[i] + ".jpg")
        gsv_imager4.append(im4)
        nrf4.append(name4[i])

# loading filtered output mask files
m1f = []
m2f = []
m3f = []
m4f = []
for i in range(0,792):
    m1 = np.load('/home/yelhe/scratch/gsvrp_of/' + nrf1[i] + 'f.npy', allow_pickle=True)
    m1f.append(m1)
    m2 = np.load('/home/yelhe/scratch/gsvrp_of/' + nrf2[i] + 'f.npy', allow_pickle=True)
    m2f.append(m2)
    m3 = np.load('/home/yelhe/scratch/gsvrp_of/' + nrf3[i] + 'f.npy', allow_pickle=True)
    m3f.append(m3)
    m4 = np.load('/home/yelhe/scratch/gsvrp_of/' + nrf4[i] + 'f.npy', allow_pickle=True)
    m4f.append(m4)

In [ ]:
# applying written functions to calculate mask properties
dfp = pd.DataFrame()
dfp1 = []
dfp2 = []
dfp3 = []
dfp4 = []
for i in range(0, 792):
    dfp1 = feature_summary(gsv_imager1[i], m1f[i])
    dfp1 = dfp1.assign(Nr = nrf1[i])
    
    dfp2 = feature_summary(gsv_imager2[i], m2f[i])
    dfp2 = dfp2.assign(Nr = nrf2[i])
    
    dfp3 = feature_summary(gsv_imager3[i], m3f[i])
    dfp3 = dfp3.assign(Nr = nrf3[i])
    
    dfp4 = feature_summary(gsv_imager4[i], m4f[i])
    dfp4 = dfp4.assign(Nr = nrf4[i])
    
    dfp = pd.concat([dfp, dfp1, dfp2, dfp3, dfp4])
    print(i, len(dfp.index))    
# saving mask properties of all random pseudo locations    
dfp.to_csv('/home/yelhe/scratch/df/df_pseudo_0_792_new.csv', index = False)

### Prediction of object (label) group of masks - curb extraction

In [ ]:
# loading files of calculated mask properites
df_a = pd.read_csv('/home/yelhe/scratch/df/df_accident_0_295.csv')
df_p = pd.read_csv('/home/yelhe/scratch/df/df_pseudo_0_792_new.csv')
# preparing for classification model
dfa = df_a
dfp = df_p
dfa = dfa.drop(['mask', 'Nr','isconvex','is_cen_inside','is_mce_inside', 'leftm', 'rightm'], axis = 1)
dfp = dfp.drop(['mask', 'Nr','isconvex','is_cen_inside','is_mce_inside', 'leftm', 'rightm'], axis = 1)
# selecting only the masks located in the lower part
dfa = dfa.loc[dfa['topm'] >= 300]
dfp = dfp.loc[dfp['topm'] >= 300]
dfa = dfa.drop(['topm'], axis = 1)
dfp = dfp.drop(['topm'], axis = 1)

In [ ]:
# loading trained RF classification model for curb extraction
rf = joblib.load("my_random_forest_1.joblib")
y_pred_a = rf.predict(dfa)
y_pred_p = rf.predict(dfp)
dfal = dfa
dfal['p'] = y_pred_a
dfpl = dfp
dfpl['p'] = y_pred_p
dfal.to_csv('/home/yelhe/script/mt/output/pred_accident.csv', index = False)
dfpl.to_csv('/home/yelhe/script/mt/output/pred_pseudo.csv', index = False)
dfa0 = df_a
dfp0 = df_p
dfa0 = dfa0.loc[dfa0['topm'] >= 300]
dfp0 = dfp0.loc[dfp0['topm'] >= 300]
dfa_pl = pd.merge(dfa0, dfal, on = ['gmedian', 'rmedian', 'bmedian', 'gmean', 
                                   'rmean', 'bmean', 'gstd', 'rstd', 'bstd', 
                                   'gq25', 'gq75', 'rq25', 'rq75', 'bq25', 
                                   'bq75', 'cdmean', 'cdstd', 'area', 'aspect_ratio_wh_s', 
                                   'extent_s', 'solidity', 'aspect_ratio_wh', 'extent', 
                                   'orien_rre', 'orien_ell', 'ed', 'ratio_ell', 'perimeter', 'bottomm'])
dfp_pl = pd.merge(dfp0, dfpl, on = ['gmedian', 'rmedian', 'bmedian', 'gmean', 
                                   'rmean', 'bmean', 'gstd', 'rstd', 'bstd', 
                                   'gq25', 'gq75', 'rq25', 'rq75', 'bq25', 
                                   'bq75', 'cdmean', 'cdstd', 'area', 'aspect_ratio_wh_s', 
                                   'extent_s', 'solidity', 'aspect_ratio_wh', 'extent', 
                                   'orien_rre', 'orien_ell', 'ed', 'ratio_ell', 'perimeter', 'bottomm'])

# saving predicted label of masks into files
dfa_pl.to_csv('/home/yelhe/script/mt/output/pred_accident_label.csv', index = False)
dfp_pl.to_csv('/home/yelhe/script/mt/output/pred_pseudo_label.csv', index = False)

### Prediction of object (label) group of masks - whole scene entropy

In [ ]:
# preparing
dfae = df_a
dfpe = df_p
dfae = dfae.drop(['mask', 'Nr','isconvex','is_cen_inside','is_mce_inside'], axis = 1)
dfpe = dfpe.drop(['mask', 'Nr','isconvex','is_cen_inside','is_mce_inside'], axis = 1)
# loading trained RF classification model for whole scene entropy
rfe = joblib.load("my_random_forest_2_whole7.joblib")
y_pred_aw = rfe.predict(dfae)
print(np.unique(y_pred_aw, return_counts=True))
y_pred_pw = rfe.predict(dfpe)
print(np.unique(y_pred_pw, return_counts=True))
# saving into dataframes
dfael = dfae
dfael['p'] = y_pred_aw
dfpel = dfpe
dfpel['p'] = y_pred_pw
dfa1 = df_a
dfp1 = df_p
# dataframe_accident_forentropy_predicted_label
dfa_epl = pd.merge(dfa1, dfael, on = ['gmedian', 'rmedian', 'bmedian', 'gmean', 
                                   'rmean', 'bmean', 'gstd', 'rstd', 'bstd', 
                                   'gq25', 'gq75', 'rq25', 'rq75', 'bq25', 
                                   'bq75', 'cdmean', 'cdstd', 'area', 'aspect_ratio_wh_s', 
                                   'extent_s', 'solidity', 'aspect_ratio_wh', 'extent', 
                                   'orien_rre', 'orien_ell', 'ed', 'ratio_ell', 'perimeter', 'bottomm',
                                   'topm', 'leftm', 'rightm'])
# dataframe_pseudo_forentropy_predicted_label
dfp_epl = pd.merge(dfp1, dfpel, on = ['gmedian', 'rmedian', 'bmedian', 'gmean', 
                                   'rmean', 'bmean', 'gstd', 'rstd', 'bstd', 
                                   'gq25', 'gq75', 'rq25', 'rq75', 'bq25', 
                                   'bq75', 'cdmean', 'cdstd', 'area', 'aspect_ratio_wh_s', 
                                   'extent_s', 'solidity', 'aspect_ratio_wh', 'extent', 
                                   'orien_rre', 'orien_ell', 'ed', 'ratio_ell', 'perimeter', 'bottomm',
                                   'topm', 'leftm', 'rightm'])
dfa_epl.groupby(['p']).size().reset_index().rename(columns={0:'count'})
# p count 
# 0 5957
# 1 316
# 2 9490
# 3 1655
# 4 30879
# 5 1798
# 6 7239

### Prediction of object (label) group of masks - ground scene entropy

In [ ]:
# preparing (using the infrastructure groups recognized by model above)
dfai = dfa_epl
dfpi = dfp_epl
# label: 1 curb, 2 infrastructure
dfai = dfa_epl.loc[(dfa_epl['p'] == 1)|(dfa_epl['p'] == 2)]
dfpi = dfp_epl.loc[(dfp_epl['p'] == 1)|(dfp_epl['p'] == 2)]
# loading trained classification model for ground scene entropy
rfi = joblib.load("my_random_forest_3_ground7_new1.joblib")
# preparing for prediction
dfai = dfai.drop(['mask', 'Nr','isconvex','is_cen_inside','is_mce_inside', 'p'], axis = 1)
dfpi = dfpi.drop(['mask', 'Nr','isconvex','is_cen_inside','is_mce_inside', 'p'], axis = 1)
y_pred_ai = rfi.predict(dfai)
print(np.unique(y_pred_ai, return_counts=True))
y_pred_pi = rfi.predict(dfpi)
print(np.unique(y_pred_pi, return_counts=True))
# saving into dataframes
dfail = dfai
dfail['p'] = y_pred_ai
dfpil = dfpi
dfpil['p'] = y_pred_pi
dfa1 = df_a
dfp1 = df_p
# dataframe_accident_forentropy_infrastructure_ground_predicted_label
dfa_eipl = pd.merge(dfa1, dfail, on = ['gmedian', 'rmedian', 'bmedian', 'gmean', 
                                   'rmean', 'bmean', 'gstd', 'rstd', 'bstd', 
                                   'gq25', 'gq75', 'rq25', 'rq75', 'bq25', 
                                   'bq75', 'cdmean', 'cdstd', 'area', 'aspect_ratio_wh_s', 
                                   'extent_s', 'solidity', 'aspect_ratio_wh', 'extent', 
                                   'orien_rre', 'orien_ell', 'ed', 'ratio_ell', 'perimeter', 'bottomm',
                                   'topm', 'leftm', 'rightm'])
# dataframe_pseudo_forentropy_infrastructure_ground_predicted_label
dfp_eipl = pd.merge(dfp1, dfpil, on = ['gmedian', 'rmedian', 'bmedian', 'gmean', 
                                   'rmean', 'bmean', 'gstd', 'rstd', 'bstd', 
                                   'gq25', 'gq75', 'rq25', 'rq75', 'bq25', 
                                   'bq75', 'cdmean', 'cdstd', 'area', 'aspect_ratio_wh_s', 
                                   'extent_s', 'solidity', 'aspect_ratio_wh', 'extent', 
                                   'orien_rre', 'orien_ell', 'ed', 'ratio_ell', 'perimeter', 'bottomm',
                                   'topm', 'leftm', 'rightm'])

### Calculation of entropy variables

In [ ]:
# calculating whole scene entropy
awe = dfa_epl[['Nr', 'p']]
pwe = dfp_epl[['Nr', 'p']]
awe.loc[:,'Nr'] = awe.loc[:,'Nr'].values.astype(str)
awe.loc[:,'id'] = awe.Nr.str[-15:-1]
awe.loc[:,'dir'] = awe.Nr.str[-1]
pwe.loc[:,'Nr'] = pwe.loc[:,'Nr'].values.astype(str)
pwe.loc[:,'Nr'] = pwe.loc[:,'Nr'].str.zfill(5)
pwe.loc[:,'id'] = pwe.Nr.str[-5:-1]
pwe.loc[:,'dir'] = pwe.Nr.str[-1]
# predicted curb in accident points summary
awes = awe.groupby(['id', 'dir', 'p']).size().reset_index().rename(columns={0:'count'})
# predicted curb in pseudo points summary
pwes = pwe.groupby(['id', 'dir','p']).size().reset_index().rename(columns={0:'count'})
# applying written functions to calculate whole scene entropy
dfentropya = entropy_calculate(awes)
dfentropyp = entropy_calculate(pwes)
dfentropya.to_csv('/home/yelhe/script/mt/output/accident_predmaskgroup_entropy.csv', index = False)
dfentropyp.to_csv('/home/yelhe/script/mt/output/pseudo_predmaskgroup_entropy.csv', index = False)

In [ ]:
# calculating ground scene entropy
aie = dfa_eipl.loc[:, ['Nr','p']]
pie = dfp_eipl.loc[:, ['Nr','p']]
aie.loc[:,'Nr'] = aie.loc[:,'Nr'].values.astype(str)
aie.loc[:,'id'] = aie.Nr.str[-15:-1]
aie.loc[:,'dir'] = aie.Nr.str[-1]
pie.loc[:,'Nr'] = pie.loc[:,'Nr'].values.astype(str)
pie.loc[:,'Nr'] = pie.loc[:,'Nr'].str.zfill(5)
pie.loc[:,'id'] = pie.Nr.str[-5:-1]
pie.loc[:,'dir'] = pie.Nr.str[-1]
# predicted curb in accident points summary in ground_infrastructure scene
aies = aie.groupby(['id', 'dir', 'p']).size().reset_index().rename(columns={0:'count'})
# predicted curb in pseudo points summary in ground_infrastructure scene
pies = pie.groupby(['id', 'dir', 'p']).size().reset_index().rename(columns={0:'count'})
# applying written functions to calculate ground scene entropy
dfentropya_i = entropy_calculate(aies)
dfentropyp_i = entropy_calculate(pies)
dfentropya_i.to_csv('/home/yelhe/script/mt/output/accident_predmaskgroup_gi_entropy_new.csv', index = False)
dfentropyp_i.to_csv('/home/yelhe/script/mt/output/pseudo_predmaskgroup_gi_entropy_new.csv', index = False)

In [ ]:
# calculating image entropy
# applying written function to calculate image entropy for accident locations
en1 = []
en2 = []
en3 = []
en4 = []
for i in range(0,295):
    e1 = entropy_img(gsv_image1[i])
    en1.append(e1)
    e2 = entropy_img(gsv_image2[i])
    en2.append(e2)
    e3 = entropy_img(gsv_image3[i])
    en3.append(e3)
    e4 = entropy_img(gsv_image4[i])
    en4.append(e4)
dfe1 = pd.DataFrame()
dfe2 = pd.DataFrame()
dfe3 = pd.DataFrame()
dfe4 = pd.DataFrame()
dfe1 = dfe1.assign(Nr = n1)
dfe1 = dfe1.assign(entropy = en1)
dfe2 = dfe2.assign(Nr = n2)
dfe2 = dfe2.assign(entropy = en2)
dfe3 = dfe3.assign(Nr = n3)
dfe3 = dfe3.assign(entropy = en3)
dfe4 = dfe4.assign(Nr = n4)
dfe4 = dfe4.assign(entropy = en4)
dfe = pd.concat([dfe1, dfe2, dfe3, dfe4])
dfe.loc[:,'Nr'] = dfe.loc[:,'Nr'].values.astype(str)
dfe.loc[:,'id'] = dfe.Nr.str[-15:-1]
dfe.loc[:,'dir'] = dfe.Nr.str[-1]
dfe.to_csv('/home/yelhe/script/mt/output/accident_image_entropy.csv', index = False)

# applying written function to calculate image entropy for random pseudo locations
enr1 = []
enr2 = []
enr3 = []
enr4 = []
for i in range(0,792):
    e1 = entropy_img(gsv_imager1[i])
    enr1.append(e1)
    e2 = entropy_img(gsv_imager2[i])
    enr2.append(e2)
    e3 = entropy_img(gsv_imager3[i])
    enr3.append(e3)
    e4 = entropy_img(gsv_imager4[i])
    enr4.append(e4)
dfer1 = pd.DataFrame()
dfer2 = pd.DataFrame()
dfer3 = pd.DataFrame()
dfer4 = pd.DataFrame()
dfer1 = dfer1.assign(Nr = nrf1)
dfer1 = dfer1.assign(entropy = enr1)
dfer2 = dfer2.assign(Nr = nrf2)
dfer2 = dfer2.assign(entropy = enr2)
dfer3 = dfer3.assign(Nr = nrf3)
dfer3 = dfer3.assign(entropy = enr3)
dfer4 = dfer4.assign(Nr = nrf4)
dfer4 = dfer4.assign(entropy = enr4)
dfer = pd.concat([dfer1, dfer2, dfer3, dfer4])
dfer.loc[:,'Nr'] = dfer.loc[:,'Nr'].values.astype(str)
dfer.loc[:,'id'] = dfer.Nr.str[-5:-1]
dfer.loc[:,'dir'] = dfer.Nr.str[-1]
dfer.to_csv('/home/yelhe/script/mt/output/pseudo_image_entropy.csv', index = False)

### Summary of entropy variables

In [ ]:
# loading files of calculated entropy 
# image entropy
iea = pd.read_csv('/home/yelhe/script/mt/output/accident_image_entropy.csv')
iep = pd.read_csv('/home/yelhe/script/mt/output/pseudo_image_entropy.csv')
# mask entropy of whole scene
dfmwa = pd.read_csv('/home/yelhe/script/mt/output/accident_predmaskgroup_entropy.csv')
dfmwp = pd.read_csv('/home/yelhe/script/mt/output/pseudo_predmaskgroup_entropy.csv')
# mask entropy of ground scene
dfmga = pd.read_csv('/home/yelhe/script/mt/output/accident_predmaskgroup_gi_entropy_new.csv')
dfmgp = pd.read_csv('/home/yelhe/script/mt/output/pseudo_predmaskgroup_gi_entropy_new.csv')

In [ ]:
# accident locations
# for image entropy
iea['Nr'] = iea['Nr'].values.astype(str)
iea['id'] = iea['id'].values.astype(str)
iea.rename(columns = {'entropy':'ie'}, inplace = True)
# for whole scene entropy
mwa = dfmwa.drop(['p', 'count', 'sum', 'percentage'], axis = 1)
mwa = mwa.drop_duplicates()
mwa['id'] = mwa['id'].values.astype(str)
mwa.rename(columns = {'entropy':'mew', 'id_dir':'Nr'}, inplace = True)
# for ground scene entropy
mga = dfmga.drop(['p', 'count', 'sum', 'percentage'], axis = 1)
mga = mga.drop_duplicates()
mga['id'] = mga['id'].values.astype(str)
mga.rename(columns = {'entropy':'meg', 'id_dir':'Nr'}, inplace = True)
# summarising
df_me = pd.merge(mwa, mga, on = ['id', 'dir', 'Nr'])
df_me['Nr'] = df_me['Nr'].values.astype(str)
df_me['id'] = df_me['id'].values.astype(str)
df_me['dir'] = df_me['dir'].values.astype(str)
iea['Nr'] = iea['Nr'].values.astype(str)
iea['id'] = iea['id'].values.astype(str)
iea['dir'] = iea['dir'].values.astype(str)
df_en_a = pd.merge(df_me, iea, on = ['id', 'dir', 'Nr'])
# saving into file
df_en_a.to_csv('/home/yelhe/script/mt/output/accident_entropy_sum_new.csv', index = False)

# random pseudo locations
# for image scene entropy
iep['Nr'] = iep['Nr'].values.astype(str)
iep['Nr'] = iep['Nr'].str.zfill(5)
iep['id'] = iep['id'].values.astype(str)
iep['id'] = iep['id'].str.zfill(4)
iep.rename(columns = {'entropy':'ie'}, inplace = True)
# for whole scene entropy
mwp = dfmwp.drop(['p', 'count', 'sum', 'percentage'], axis = 1)
mwp = mwp.drop_duplicates()
mwp.rename(columns = {'entropy':'mew', 'id_dir':'Nr'}, inplace = True)
mwp['Nr'] = mwp['Nr'].values.astype(str)
mwp['Nr'] = mwp['Nr'].str.zfill(5)
mwp['id'] = mwp['id'].values.astype(str)
mwp['id'] = mwp['id'].str.zfill(4)
# for ground scene entropy
mgp = dfmgp.drop(['p', 'count', 'sum', 'percentage'], axis = 1)
mgp = mgp.drop_duplicates()
mgp.rename(columns = {'entropy':'meg', 'id_dir':'Nr'}, inplace = True)
mgp['Nr'] = mgp['Nr'].values.astype(str)
mgp['Nr'] = mgp['Nr'].str.zfill(5)
mgp['id'] = mgp['id'].values.astype(str)
mgp['id'] = mgp['id'].str.zfill(4)
# summarising
df_me_p = pd.merge(mwp, mgp, on = ['id', 'dir', 'Nr'])
df_en_p = pd.merge(df_me_p, iep, on = ['id', 'dir', 'Nr'])
# saving into file
df_en_p.to_csv('/home/yelhe/script/mt/output/pseudo_entropy_sum_new.csv', index = False)